In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import re
from sqlalchemy import create_engine

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url = 'https://www.imdb.com/list/ls020366512/'
browser.visit(url)

In [4]:
html = browser.html
soup = bs(html, 'html.parser')

# scrapping: Title / Cast / Revenue / Year


In [5]:
# title_list = soup.find_all("h3", class_="lister-item-header")
# title_list
title_list = soup.find_all("div", class_="lister-item-content")
title_list

[<div class="lister-item-content">
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">1.</span>
 <a href="/title/tt4154756/?ref_=ttls_li_tt">Avengers: Infinity War</a>
 <span class="lister-item-year text-muted unbold">(2018)</span>
 </h3>
 <p class="text-muted text-small">
 <span class="certificate">PG-13</span>
 <span class="ghost">|</span>
 <span class="runtime">149 min</span>
 <span class="ghost">|</span>
 <span class="genre">
 Action, Adventure, Sci-Fi            </span>
 </p>
 <div class="ipl-rating-widget">
 <div class="ipl-rating-star small">
 <span class="ipl-rating-star__star">
 <svg class="ipl-icon ipl-star-icon " fill="#000000" height="24" viewbox="0 0 24 24" width="24" xmlns="http://www.w3.org/2000/svg">
 <path d="M0 0h24v24H0z" fill="none"></path>
 <path d="M12 17.27L18.18 21l-1.64-7.03L22 9.24l-7.19-.61L12 2 9.19 8.63 2 9.24l5.46 4.73L5.82 21z"></path>
 <path d="M0 0h24v24H0z" fill="none"></path>
 </svg>
 </span>
 <span class="ipl-rating

In [6]:
movies = []
for x in title_list:
    try:
        movie_dict = {}
        movie_dict["title"] = x.a.text
#         print(movie_dict["title"])

        regex = r'.*([0-9]{4}).*'
        match = re.match(regex, x.find("span", class_="lister-item-year").text)
        movie_dict["year"] = match.group(1)
#         print(movie_dict["year"])

        start_stars = x.text.strip().find("Stars:") + len("Stars:")
        end_stars = x.text.strip().find("Votes:")
        stars = x.text.strip()[start_stars:end_stars]
        stars = [star.strip() for star in stars.split(',')]
        movie_dict["stars"] = ','.join(stars)
#         print(movie_dict["stars"])

        movie_dict["revenue"] = x.find("span",{"name":"nv"}).text
#         print(movie_dict["revenue"])


        movies.append(movie_dict)
    
        
    except AttributeError as e:
#         print(x)
        print(e)
        
print(movies)
browser.quit()

'NoneType' object has no attribute 'text'
[{'title': 'Avengers: Infinity War', 'year': '2018', 'stars': 'Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Chris Evans', 'revenue': '646,761'}, {'title': 'Solo: A Star Wars Story', 'year': '2018', 'stars': 'Alden Ehrenreich,Woody Harrelson,Emilia Clarke,Donald Glover', 'revenue': '220,555'}, {'title': 'Black Panther', 'year': '2018', 'stars': "Chadwick Boseman,Michael B. Jordan,Lupita Nyong'o,Danai Gurira", 'revenue': '503,534'}, {'title': 'Deadpool 2', 'year': '2018', 'stars': 'Ryan Reynolds,Josh Brolin,Morena Baccarin,Julian Dennison', 'revenue': '380,614'}, {'title': 'Ant-Man and the Wasp', 'year': '2018', 'stars': 'Paul Rudd,Evangeline Lilly,Michael Peña,Walton Goggins', 'revenue': '232,356'}, {'title': 'Aquaman', 'year': '2018', 'stars': 'Jason Momoa,Amber Heard,Willem Dafoe,Patrick Wilson', 'revenue': '246,134'}, {'title': 'Jurassic World: Fallen Kingdom', 'year': '2018', 'stars': 'Chris Pratt,Bryce Dallas Howard,Rafe Spall,Justice Smi

In [11]:
movie_df = pd.DataFrame(movies)
movie_df

,revenue,stars,title,year
0,"646,761","Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo...",Avengers: Infinity War,2018
1,"220,555","Alden Ehrenreich,Woody Harrelson,Emilia Clarke...",Solo: A Star Wars Story,2018
2,"503,534","Chadwick Boseman,Michael B. Jordan,Lupita Nyon...",Black Panther,2018
3,"380,614","Ryan Reynolds,Josh Brolin,Morena Baccarin,Juli...",Deadpool 2,2018
4,"232,356","Paul Rudd,Evangeline Lilly,Michael Peña,Walton...",Ant-Man and the Wasp,2018
5,"246,134","Jason Momoa,Amber Heard,Willem Dafoe,Patrick W...",Aquaman,2018
6,"214,377","Chris Pratt,Bryce Dallas Howard,Rafe Spall,Jus...",Jurassic World: Fallen Kingdom,2018
7,"227,989","Tom Cruise,Henry Cavill,Ving Rhames,Simon Pegg",Mission: Impossible - Fallout,2018
8,"296,556","Tye Sheridan,Olivia Cooke,Ben Mendelsohn,Lena ...",Ready Player One,2018
9,"90,669","Boyd Holbrook,Trevante Rhodes,Jacob Tremblay,K...",The Predator,2018


In [8]:
movie_df.to_csv("new_release.csv", encoding="utf-8", index=False, header=True)

# Convert to SQL DB

In [9]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///new_release.sqlite', echo=False)

movie_df.to_sql('new_release', con=engine, if_exists='replace')
engine.execute("SELECT * FROM new_release").fetchall()

[(0, '646,761', 'Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Chris Evans', 'Avengers: Infinity War', '2018'),
 (1, '220,555', 'Alden Ehrenreich,Woody Harrelson,Emilia Clarke,Donald Glover', 'Solo: A Star Wars Story', '2018'),
 (2, '503,534', "Chadwick Boseman,Michael B. Jordan,Lupita Nyong'o,Danai Gurira", 'Black Panther', '2018'),
 (3, '380,614', 'Ryan Reynolds,Josh Brolin,Morena Baccarin,Julian Dennison', 'Deadpool 2', '2018'),
 (4, '232,356', 'Paul Rudd,Evangeline Lilly,Michael Peña,Walton Goggins', 'Ant-Man and the Wasp', '2018')]

In [10]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
engine = create_engine("sqlite:///new_release.sqlite")
 
Base = automap_base()
# use this superclass instead of using the declarative base we used before. This one can be used to reflect table schema 
 
Base.prepare(engine, reflect=True)
# prepares the engine, copy schemas of all tables in the DB into Base 
 
engine.execute("SELECT * FROM new_release").fetchall()
# print(Base.classes.keys())
# optional, returns names of all the tables in Base 

# Base.classes
# new_release = Base.classes.new_release
# associates a table with a class. This class will be used to query the table.
 
# movie_obj = Session(engine)
# # creates a session engine
 
# first_row = movie_obj.query(new_release).first()
# first_row.__dict__
# # optional: prints the first row of the table associated with the class in dictionary form
 
# result = movie_obj.query(new_release.column_name).all()
# for row in result:
#     print(row)
    
    
# # movie_df.to_sql(name='new_release', con=engine, if_exists='append', index=False)

[(0, '646,761', 'Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Chris Evans', 'Avengers: Infinity War', '2018'),
 (1, '220,555', 'Alden Ehrenreich,Woody Harrelson,Emilia Clarke,Donald Glover', 'Solo: A Star Wars Story', '2018'),
 (2, '503,534', "Chadwick Boseman,Michael B. Jordan,Lupita Nyong'o,Danai Gurira", 'Black Panther', '2018'),
 (3, '380,614', 'Ryan Reynolds,Josh Brolin,Morena Baccarin,Julian Dennison', 'Deadpool 2', '2018'),
 (4, '232,356', 'Paul Rudd,Evangeline Lilly,Michael Peña,Walton Goggins', 'Ant-Man and the Wasp', '2018')]